# Photometry Example: Doing One Catalogue In Full

Owner: Aaron E. Watkins

Last verified to run: 30 August 2021

Verified Stack release: w_2021_33

In [ ]:
# Checking what version of the Stack I'm using
! echo $HOSTNAME
! eups list lsst_distrib -s

**Summary:**

This Notebook can be run to produce Python pickle files containing photometry of injected models (which depends on the catalogue loaded in prior to image retrieval and photometry).  See codes `image_retrieval.py` and `surface_photometry.py` for documentation on the software used herein.  One must be logged into the Rubin Science Platform Notebook aspect, given the hard dependence on directories thereon.

### Python imports and utility function definitions

In [ ]:
import image_retrieval as imret # This module
import surface_photometry as surfphot # This module

from lsst.daf.persistence import Butler # NOTE: will upgrade to Gen. 3 Butler when new injection is available
import lsst.geom as geom

from astropy.table import Table
import numpy as np
import os
import pickle

In [ ]:
def getMaxR(stampWidth, pa, ell):
    '''
    Uses the stamps themselves to determine the maximum aperture radius
    '''
    if (ell == 0.5):
        if (pa == 0) | (pa == 90) | (pa == 180):
            return 0.5*stampWidth
        else:
            return np.sqrt((0.5*stampWidth)**2 + (0.5*stampWidth)**2)
    else:
        return 0.5*stampWidth

### Reading in table and setting up Butler

In [ ]:
# Change these directories to the appropriate one for the catalogue you want to use
# This example uses the single Sersic profile models developed as part of WP 3.7.1. and 3.7.2
catDir = '/home/lskelvin/fakes_lsstuk/input/lsstuk_lsb_sersic.fits'
fksrepo = '/home/lskelvin/fakes_lsstuk/output/lsstuk_lsb_sersic'
repo = '/datasets/hsc/repo/rerun/RC/w_2020_42/DM-27244'

# ICL + NH dwarfs are found at the following locations
# catDir = '/home/lskelvin/fakes_lsstuk/input/lsstuk_icl_dwarfs.fits'
# fksrepo = '/home/lskelvin/fakes_lsstuk/output/lsstuk_icl_dwarfs'

In [ ]:
fullCat = Table.read(catDir)
ra = fullCat['raJ2000']
dec = fullCat['decJ2000']

In [ ]:
butler = Butler(repo)
butler_fk = Butler(fksrepo)
tract = 9615
filters = ['g', 'r', 'i', 'z', 'y']

## Pre-sky-subtraction photometry of catalogue models

We want to loop over all of the patches with models actually present in them, so first we identify and store which patches have models.

The loop that follows is based on the image retrieval mechanisms established in `image_retrieval.py`, but modified to work faster for the kind of photometry we do to produce the metrics (namely, by minimizing the number of Butler calls).  We subtract images of the CCD without models on them from the same images with the models present, and do photometry on these "difference" images to conduct the photometry down to infinite depth.  Using the functions in `image_retrieval.py` would slow the code down substantially, as it would need to run the Butler several times for each model, rather than once on a patch-by-patch basis like is done below.  Those specific cutout-grabbing codes are for the community at large to use and modify as it pleases.

In [ ]:
skymap = butler_fk.get('deepCoadd_skyMap', immediate=True)
tractInfo = skymap[tract]    # Get world coordinate system

patchIds = []
allPos = []
for i in range(len(ra)):
    pos = geom.SpherePoint(ra[i], dec[i], geom.radians)
    patchInfo = tractInfo.findPatch(pos)
    patchIds.append(patchInfo.getIndex()) # For finding CCDs and Visit info
    allPos.append(pos) # For checking CCD WCS pixel coordinates later
patchIds = np.unique(patchIds, axis=0)
print(str(len(patchIds))+' found')

In [ ]:
# Suppress Numpy log errors for notebook cleanliness
np.seterr(divide='ignore')
np.seterr(invalid='ignore')

The main loop.  Please note that this takes a very long time to run!

In [ ]:
# Setting up main dictionary; main keys are galaxy indexes
mainDict = {}
for i in range(len(allPos)):
    mainDict[str(i)] = {}

modelInds = list(range(len(allPos)))
# Loop over all filters, and for each filter, loop over all patches
for filt in filters:
    # If re-starting from a crash, load in pickle file as main dictionary
    # This is written to pick up where the loops left off at the crash time
    if os.path.exists('mags.p'):
        mainDict = pickle.load(open('mags.p', 'rb'))
        keepInds = [int(key) for key in mainDict if filt+'Prof' in mainDict[key]]
        if len(keepInds)==len(allPos):
            continue
    else:
        # Want this remade for every filter
        keepInds = [] # For keeping track of models that have had photometry done on them
    print('Doing filter ',filt)
    for pch in patchIds:
        if len(keepInds)==len(allPos):
            print('Found all models.  Moving to next filter....')
            break
        pch = str(pch[0])+','+str(pch[1])
        print('Doing patch '+pch, end=' | ')
        coaddId = dict(tract=tract, patch=pch, filter="HSC-"+filt.upper())
        exp = butler_fk.get('fakes_deepCoadd', coaddId, immediate=True)
        ccds, visits = imret.getCcdVisit(exp)
        
        # For each CCD and Visit, checks to see if models are on it
        # If they are, it does photometry and records it in the main dictionary
        for j in range(len(ccds)):
            # Stop looking once the last model is found
            if len(keepInds)==len(allPos):
                break
            # First check if any of the models are in the specific visit
            wcs = butler_fk.get('calexp_wcs', tract=tract, visit=visits[j], ccd=ccds[j])
            for k in modelInds:
                # Skip models already done
                if k in keepInds:
                    continue
                x, y = imret.getXyCoord(wcs, allPos[k])
                # First time a coordinate is found in the image, break this and continue
                want = (x < 2048) & (x > 0) & (y < 4176) & (y > 0)
                if want:
                    brk = 0 # Allows it to move on to grabbing calexps
                    break
                else:
                    brk = 1 # Otherwise, jump to next j loop
            # If the coordinate wasn't found on any CCD or Visit, skip to the next CCD, Visit
            if brk:
                continue
            # Otherwise, grab the exposures for photometry
            fk_calexp = butler_fk.get('fakes_calexp', tract=tract, visit=visits[j], ccd=ccds[j])
            calexp = butler_fk.get('calexp', tract=tract, visit=visits[j], ccd=ccds[j])
            wcs = fk_calexp.getWcs()
            magZp = calexp.getPhotoCalib().instFluxToMagnitude(1)
            
            # Making the "difference" image and doing photometry on it
            imArr = fk_calexp.image.array - calexp.image.array
            for k in modelInds:
                # Skip models already done
                if k in keepInds:
                    continue
                x, y = imret.getXyCoord(wcs, allPos[k])
                # Preliminary check; coordinate must be within 1 Reff of the CCD edge
                hlr_px = fullCat['BulgeHalfLightRadius'][k]/0.167
                boundX = (x - hlr_px <= 0) | (x + hlr_px >= 2048)
                boundY = (y - hlr_px <= 0) | (y + hlr_px >= 4176)
                bounds = boundX | boundY # True means it's outside the CCD edge
                if bounds:
                    # Skip to next model
                    continue
                # Otherwise do the photometry and record that the model is done
                # Calculate aperture size based on injected stamp dimensions
                maxR = getMaxR(fullCat['stampWidth'][k], 
                               fullCat['pa_bulge'][k],
                               fullCat['b_b'][k])
                # Using 1 pixel radial bins, so the radius array is just range(len(meanI))
                photDict = surfphot.surfBriProfs(imArr, x, y, 
                                                 180 - fullCat['pa_bulge'][k], # Note that orientation is different in calexps
                                                 1 - fullCat['b_b'][k], 
                                                 1, maxR, magZp=magZp)
                totFlux = np.nanmax(photDict['cog']) # Taking brightest value as magnitude
                sbProf = photDict['meanI']
                
                # Finally storing the output as magnitudes
                mainDict[str(k)][filt+'magVarMeas'] = -2.5*np.log10(totFlux) + magZp
                mainDict[str(k)][filt+'Prof'] = -2.5*np.log10(sbProf) + magZp + 2.5*np.log10(0.168**2)
                mainDict[str(k)][filt+'ccd'] = ccds[j] # For debugging
                mainDict[str(k)][filt+'visit'] = visits[j] # For debugging
                mainDict[str(k)][filt+'x'] = x # For debugging
                mainDict[str(k)][filt+'y'] = y # For debugging
                mainDict[str(k)][filt+'maxR'] = maxR # For debugging
                    
                keepInds.append(k) # If model is finished, record it here
                pickle.dump(mainDict, open('mags.p', 'wb')) # Save progress to a pickle dump

## Now getting sky-subtracted magnitudes

This is the same principle as the above, but runs on the coadd images instead of individual calexps.  Here we retrieve whole patches and do photometry of all models on each patch at a time, to avoid having to retrieve stamps for every model individually.

Note that this takes an additional long time to run.  Best practice is to leave these running overnight and analyze the photometry in the morning.

In [ ]:
# Setting up main dictionary; main keys are galaxy indexes
mainDict = {}
for i in range(len(allPos)):
    mainDict[str(i)] = {}
    
modelInds = list(range(len(allPos)))
# Loop over all filters, and for each filter, loop over all patches
for filt in filters:
    # If re-starting from a crash, load in pickle file as main dictionary
    if os.path.exists('coadd_mags.p'):
        mainDict = pickle.load(open('coadd_mags.p', 'rb'))
        keepInds = [int(key) for key in mainDict if filt+'Prof' in mainDict[key]]
        if len(keepInds)==len(allPos):
            continue
    else:
        # Want this remade for every filter
        keepInds = [] # For keeping track of models that have completed photometry
    print('Doing filter ',filt)
    for pch in patchIds:
        if len(keepInds)==len(allPos):
            print('Found all models.  Moving to next filter....')
            break
        pch = str(pch[0])+','+str(pch[1])
        print('Doing patch '+pch, end=' | ')
        coaddId = dict(tract=tract, patch=pch, filter="HSC-"+filt.upper())
        fk_exp = butler_fk.get('fakes_deepCoadd', coaddId, immediate=True)
        exp = butler.get('deepCoadd', coaddId, immediate=True)
        magZp = exp.getPhotoCalib().instFluxToMagnitude(1)
        
        # "Difference" image is created here
        imArr = fk_exp.image.array - exp.image.array
        for k in modelInds:
            # Skip models already done
            if k in keepInds:
                continue
            # This requires a different method to derive the proper (x,y) coordinate than the visit-level calexps
            xy = fk_exp.getWcs().skyToPixel(allPos[k]) - fk_exp.getBBox().getMin()
            x = xy[0]
            y = xy[1]
            # Preliminary check; if coordinate not in image, skip loop
            xmax = fk_exp.image.array.shape[1]
            ymax = fk_exp.image.array.shape[0]
            want = (x < xmax) & (x > 0) & (y < ymax) & (y > 0)
            if not want:
                continue
            # Calculate maximum radius now, in exactly the same way as the previous loop
            maxR = getMaxR(fullCat['stampWidth'][k], 
                           fullCat['pa_bulge'][k],
                           fullCat['b_b'][k])
            # Check now whether coordinate is within 1 Reff of the patch edge
            # This should never be true for our two catalogues, but for the sake of robustness, we include this check.
            hlr_px = fullCat['BulgeHalfLightRadius'][k]/0.167
            boundX = (x - hlr_px <= 0) | (x + hlr_px >= exp.image.array.shape[1])
            boundY = (y - hlr_px <= 0) | (y + hlr_px >= exp.image.array.shape[0])
            bounds = boundX | boundY # True means it's outside the patch edge
            # If not outside, do photometry on patch directly
            if not bounds:
                # Using 1 pixel radial bins, so the radius array is just range(len(meanI))
                photDict = surfphot.surfBriProfs(imArr, x, y, 
                                                 fullCat['pa_bulge'][k],
                                                 1 - fullCat['b_b'][k], 
                                                 1, maxR, magZp=magZp)
                totFlux = np.nanmax(photDict['cog']) # Taking brightest value as magnitude
                sbProf = photDict['meanI']
                
                # Storing results as magnitudes
                mainDict[str(k)][filt+'magVarMeas'] = -2.5*np.log10(totFlux) + magZp
                mainDict[str(k)][filt+'Prof'] = -2.5*np.log10(sbProf) + magZp + 2.5*np.log10(0.168**2)
                mainDict[str(k)][filt+'patch'] = pch # For debugging
                mainDict[str(k)][filt+'maxR'] = maxR # For debugging
                    
                keepInds.append(k) # If model has photometry, record it here
                pickle.dump(mainDict, open('coadd_mags.p', 'wb')) # Save progress to a pickle dump
                
            # If stamp boundaries straddle two patches, grab a cutout of the full model
            else:
                fk_cut = imret.getRobustCutout(ra[k], dec[k], 2*maxR, butler_fk,
                                               filt, 'fakes_deepCoadd',
                                               tract, tractInfo)
                cut = imret.getRobustCutout(ra[k], dec[k], 2*maxR, butler,
                                            filt, 'deepCoadd',
                                            tract, tractInfo)
                x = fk_cut.image.array.shape[1]//2
                y = fk_cut.image.array.shape[0]//2
                imArr = fk_cut.image.array - cut.image.array
                photDict = surfphot.surfBriProfs(imArr, x, y, 
                                                 fullCat['pa_bulge'][k],
                                                 1 - fullCat['b_b'][k], 
                                                 1, maxR, magZp=magZp)
                totFlux = np.nanmax(photDict['cog'])
                sbProf = photDict['meanI']
                mainDict[str(k)][filt+'magVarMeas'] = -2.5*np.log10(totFlux) + magZp
                mainDict[str(k)][filt+'Prof'] = -2.5*np.log10(sbProf) + magZp + 2.5*np.log10(0.168**2)
                mainDict[str(k)][filt+'patch'] = pch
                mainDict[str(k)][filt+'maxR'] = maxR
                    
                keepInds.append(k)
                pickle.dump(mainDict, open('coadd_mags.p', 'wb'))